In [2]:
# Used for detecting camera, capturing and showing images on display
# and for resizing image into 300x300
import cv2

# Used for hand detection
from cvzone.HandTrackingModule import HandDetector

# for defining the size and colours of images
import numpy as np

# for ceil() function
import math

# for defining unique name along with file name using time()
import time

"""
provides functionality for image classification tasks, 
such as training models to classify images into 
different categories.
"""
from cvzone.ClassificationModule import Classifier

import tensorflow.keras

In [3]:
"""
It will return the video from the first webcam on our device,
1 for if we have other external webcam connected to our devices
"""
cap=cv2.VideoCapture(0)

In [4]:
"""
HandDetector() will detect the hand from Video.
maxHands is the maximum number of hands which it will detect.
'1' represents only one hand it will detect
"""
detector=HandDetector(maxHands=1)

In [5]:
"""
"Model/keras_model.h5": This is the path to a trained 
model file (in the HDF5 format) that will be used for 
image classification.

"Model/labels.txt": This is the path to a labels file
that contains the class labels corresponding to the 
trained model.
"""
classifier=Classifier("Model/keras_model.h5","Model/labels.txt")


In [6]:
# defining labels for as per labels.txt
labels=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]

In [20]:
while True:
    """
    'cap' object allows to access the camera device
    read() reads the next frame form the video
    'success' return boolean vlaues,
    True-> Frame was read successfully
    False-> no more frames to read or error occurred
    'img' matrix of pixel values represented actual frame
    """ 
    
    success, img=cap.read()
    
    """
    This line creates a copy of the img frame and assigns 
    it to the variable imgOutput
    """
    imgOutput=img.copy()
    
    """
    'detector' is instance of HandDetector
    'findHands(img)' will uses cv algo to detect the hand from 'img' object
    'hands' is list , containing hands coordinates etc
    'img' is the modified frame after hand detection algorithm applied
    """
    
    hands, img=detector.findHands(img)
    
    
    if hands:
        
        #extracting the hand coordinates from hands list
        hand=hands[0]
        
        """
        x: Represents the x-coordinate of the top-left corner of the bounding box.
        y: Represents the y-coordinate of the top-left corner of the bounding box.
        w: Represents the width of the bounding box.
        h: Represents the height of the bounding box.
        """
        
        x,y,w,h= hand['bbox']
        
        """
        imgWhite numpy array represents the white image with dimensions 300x300,
        along with 3 channel(red,green and blue).
        np.uint8 is to represent colours using 8byte (0-255)
        *255 to convert all numpy element from 1 to 255 (black to white)
        """
        imgWhite=np.ones((300,300,3),np.uint8)*255
        
        """
        Cropping the region of interest (ROI) from img
        y-20 represent starting row coordinated
        y+h+20 represents ending row coordinates
        x-20 represents starting coloumn coordinates
        x+w+20 represents ending column coordinates
        (-20,+20) pixel offset for extra space for margin/boundary
        """
        imgCrop= img[y-20:y+h+20,x-20:x+w+20]
        
        """
        The aspect ratio is a measure of how long or thin the 
        shape is along the vertical or horizontal direction.
        greater then 1 represent vertically elongated shape,
        less then 1 represents horizontally elongated shape. 
        1 represents perfect square.
        """
        
        aspectratio=h/w
        
        # if vertically elongated shape
        if aspectratio>1:
            
            k=300/h
            width_cal=math.ceil(k*w)
            imgResize=cv2.resize(imgCrop,(width_cal,300))
            imgResizeShape=imgResize.shape
            
            width_gap=math.ceil((300-width_cal)/2)
            imgWhite[:,width_gap:width_cal+width_gap]=imgResize
            
            """
            the image 'imgWhite' is provided as input to the 
            image classification model within the classifier 
            object. 
            The model then predicts the most likely class or 
            label for the image. The predicted class is stored 
            in the 'prediction' variable, while the index of the 
            predicted class within the list of labels is 
            stored in the 'index' variable.
            """
            prediction, index = classifier.getPrediction(imgWhite,draw=False)
            print(prediction,index)
            
        # horizontally elongated shape    
        else:
            
            k=300/w
            height_cal=math.ceil(k*h)
            imgResize=cv2.resize(imgCrop,(300,height_cal))
            imgResizeShape=imgResize.shape
            
            height_gap=math.ceil((300-height_cal)/2)
            imgWhite[height_gap:height_cal+height_gap,:]=imgResize
            
            prediction, index = classifier.getPrediction(imgWhite,draw=False)
            print(prediction,index)
            
        """
        This line adds text annotation to the 'imgOutput' frame. 
        It uses the cv2.putText() function from OpenCV to draw text on the image.
        """
        cv2.putText(imgOutput,labels[index],(x,y-20),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,255),2)
        
        # This line draws a rectangle around the detected hand in the imgOutput frame.
        cv2.rectangle(imgOutput,(x-20,y-20),(x+w+20,y+h+20),(255,0,255),5)    
            
        
        cv2.imshow("CroppedImage",imgCrop)
        
        cv2.imshow("Image White",imgWhite)
    
    cv2.imshow("Image",imgOutput)
    key=cv2.waitKey(1)
    
    # for exit
    if key== ord("1"):
        cap.release()
        cv2.destroyAllWindows()

    

    

1/1 [==============================] - 2s 2s/step
[1.2700743e-06, 0.0065161213, 0.00049139373, 1.1191152e-09, 3.6094889e-06, 0.00014370953, 2.108626e-11, 3.8971084e-08, 4.360311e-08, 2.2604287e-13, 0.99233705, 8.0758546e-11, 5.198894e-17, 5.6849276e-06, 2.9558836e-05, 3.2301762e-15, 4.1500176e-08, 5.574594e-06, 4.4753823e-12, 9.75844e-09, 4.8176498e-12, 3.2136423e-17, 0.0004654968, 2.2047678e-12, 6.265484e-12, 3.3671785e-07] 10
1/1 [==============================] - 0s 76ms/step
[0.17259733, 1.8175053e-08, 6.8632465e-07, 1.19929294e-14, 1.6914086e-08, 9.8125514e-15, 9.854179e-06, 7.509507e-08, 0.00014623677, 5.7598452e-05, 1.24770454e-11, 5.9112097e-09, 7.967529e-07, 6.770905e-05, 7.339882e-06, 4.974458e-11, 5.463929e-10, 7.387508e-10, 0.0048954883, 2.0363491e-07, 3.212814e-13, 7.72577e-14, 1.9023421e-06, 8.040582e-10, 0.8222086, 6.1585883e-06] 24
1/1 [==============================] - 0s 97ms/step
[0.98339635, 1.1953439e-06, 2.3272555e-08, 5.7413296e-17, 7.970581e-08, 3.8701108e-14, 1

1/1 [==============================] - 0s 75ms/step
[2.0144041e-08, 0.9998728, 3.639442e-08, 6.522744e-17, 4.3088505e-11, 5.176833e-09, 2.6064595e-12, 1.1472355e-10, 7.991195e-09, 2.2530773e-14, 7.9286583e-13, 6.074601e-18, 4.443514e-16, 2.6094082e-09, 4.748296e-11, 1.7386579e-17, 1.26747325e-14, 0.00012703381, 1.885832e-12, 2.1228067e-12, 7.06199e-13, 1.7514104e-19, 3.4881396e-09, 9.376238e-19, 4.1356043e-17, 1.3193026e-09] 1
1/1 [==============================] - 0s 72ms/step
[8.263686e-09, 0.9999902, 4.073412e-09, 3.2119518e-16, 1.2661702e-10, 1.9026571e-07, 3.2258078e-13, 7.9969135e-11, 7.1505877e-09, 1.18771765e-14, 8.202305e-12, 1.5737793e-18, 7.967258e-18, 2.767402e-09, 1.9320703e-11, 1.5245622e-18, 6.4302976e-15, 9.391086e-06, 5.2707117e-13, 6.1764894e-13, 1.2548976e-12, 2.8618572e-20, 8.2233605e-08, 1.0082014e-17, 5.412264e-18, 1.6580375e-10] 1
1/1 [==============================] - 0s 71ms/step
[1.41167e-08, 0.9996314, 7.780014e-09, 1.9624764e-15, 1.505009e-09, 1.3319776e-07,

1/1 [==============================] - 0s 81ms/step
[1.3305699e-07, 2.7608693e-11, 0.97455215, 7.731514e-17, 8.380884e-13, 3.5240206e-18, 2.2280788e-09, 8.731954e-07, 1.1123603e-11, 0.0254211, 7.243417e-16, 8.154229e-10, 8.2519904e-07, 1.7855311e-12, 1.0503668e-06, 5.4074593e-07, 9.346982e-06, 4.5229726e-07, 1.3508262e-05, 5.593467e-28, 1.5786541e-14, 8.447355e-21, 1.2619052e-15, 2.303335e-09, 2.463958e-10, 3.1548285e-11] 2
1/1 [==============================] - 0s 83ms/step
[8.830679e-05, 2.099224e-06, 0.9993311, 2.4594436e-15, 1.0117414e-12, 4.7587634e-16, 9.154822e-09, 1.4950944e-08, 1.3492986e-09, 3.5295256e-05, 1.0470623e-15, 7.4145426e-11, 1.41985975e-05, 7.634879e-18, 0.00046932403, 5.6759138e-05, 8.46457e-08, 8.383905e-07, 4.911007e-07, 9.089559e-20, 5.8069413e-16, 2.6691774e-18, 3.3864226e-16, 3.3366506e-12, 5.1871854e-07, 8.83961e-07] 2
1/1 [==============================] - 0s 73ms/step
[4.2932044e-10, 3.7356274e-09, 7.617076e-05, 2.9962645e-14, 9.67008e-12, 4.4933873e-19, 1

1/1 [==============================] - 0s 86ms/step
[0.00037110466, 0.6643608, 1.0621781e-06, 2.7465736e-09, 5.4908332e-06, 6.322393e-10, 1.425839e-07, 1.1325809e-06, 0.13610105, 1.2764231e-09, 1.3942301e-07, 1.3933898e-11, 7.169786e-14, 1.543395e-08, 1.5366646e-07, 4.88576e-11, 3.6624455e-09, 0.12832709, 7.1305245e-10, 0.0013532592, 7.623822e-09, 3.293728e-12, 9.7949426e-11, 2.6154032e-11, 1.3930047e-07, 0.069478355] 1
1/1 [==============================] - 0s 77ms/step
[8.775785e-05, 0.029698336, 1.6466479e-06, 9.290988e-09, 1.3247943e-06, 1.1326151e-10, 2.4350181e-06, 2.1925669e-06, 0.033263333, 9.687689e-09, 2.6068975e-10, 4.1675543e-12, 1.5747053e-09, 1.5567794e-11, 5.547995e-10, 1.2759123e-08, 1.6070237e-08, 0.93179476, 4.5977317e-13, 1.3975313e-07, 5.812389e-09, 1.0551748e-12, 6.5290967e-14, 1.0776917e-11, 3.670748e-08, 0.0051479796] 17
1/1 [==============================] - 0s 79ms/step
[3.606966e-05, 0.08605246, 1.059377e-06, 1.202248e-07, 5.0728264e-05, 7.807621e-10, 2.529024

1/1 [==============================] - 0s 83ms/step
[5.913914e-07, 0.00013888458, 2.886098e-09, 2.4621483e-10, 4.345733e-06, 4.669203e-09, 2.1119504e-08, 4.4725298e-10, 0.9998209, 7.708845e-11, 7.710882e-09, 1.0356529e-12, 3.2628503e-13, 3.1404143e-10, 7.569058e-12, 3.5015463e-11, 1.866163e-11, 3.1705602e-07, 3.3690707e-13, 7.787213e-07, 1.2326576e-13, 1.2335983e-13, 3.1981692e-08, 8.3302835e-12, 9.057421e-12, 3.4243403e-05] 8
1/1 [==============================] - 0s 70ms/step
[9.270554e-07, 0.0054247784, 7.817056e-09, 4.165393e-09, 3.5533314e-05, 1.903382e-09, 1.01994475e-08, 3.4009524e-09, 0.9888475, 1.7693295e-11, 1.236292e-07, 3.4049326e-12, 2.9629615e-15, 2.8428666e-08, 3.125731e-10, 4.6625585e-11, 1.2348525e-10, 1.470558e-06, 6.720179e-12, 2.9267915e-06, 1.1938238e-09, 4.881619e-13, 1.2363857e-08, 1.5225941e-12, 1.6992383e-11, 0.0056867814] 8
1/1 [==============================] - 0s 69ms/step
[5.4603775e-06, 0.105660915, 7.051624e-08, 6.262772e-08, 0.0005732581, 3.5399225e-07, 

...

1/1 [==============================] - 0s 72ms/step
[4.4752376e-08, 4.344243e-10, 5.6122286e-07, 3.7606216e-12, 4.84246e-15, 2.062647e-17, 1.2118235e-09, 3.575793e-11, 4.5239012e-05, 2.502483e-08, 6.9253375e-10, 0.9999006, 3.7788817e-10, 4.899789e-13, 9.120376e-13, 1.5994097e-08, 6.2511445e-09, 1.4144873e-15, 1.8425267e-10, 1.12397e-22, 6.187089e-13, 6.4722366e-10, 5.2241408e-06, 1.3511945e-11, 1.6183965e-07, 4.8134727e-05] 11
1/1 [==============================] - 0s 81ms/step
[1.7163891e-05, 5.3799782e-09, 0.005312815, 1.6539384e-12, 2.3408076e-18, 9.603379e-22, 6.8873187e-06, 8.304011e-11, 3.2099308e-08, 1.07285764e-10, 6.065673e-12, 0.00026793414, 3.3892113e-06, 7.2001155e-12, 4.7406426e-11, 9.673255e-07, 3.1460007e-05, 0.99410015, 3.599132e-14, 7.963694e-23, 9.046608e-19, 4.79197e-11, 1.4794015e-14, 1.816125e-16, 0.00025316718, 5.983749e-06] 17
1/1 [==============================] - 0s 86ms/step
[4.208383e-07, 1.1456256e-13, 8.9937245e-08, 1.02855136e-13, 2.9085348e-19, 3.5945126e

1/1 [==============================] - 0s 79ms/step
[7.441647e-08, 0.005781942, 2.6195212e-06, 1.7578688e-13, 3.0840085e-13, 9.537346e-12, 6.859821e-07, 3.5922274e-09, 1.13691016e-07, 5.442631e-11, 1.3597374e-12, 7.1403356e-12, 2.5542714e-13, 2.0576521e-05, 5.724343e-10, 3.2825003e-11, 5.6851034e-11, 0.0009195415, 1.086897e-12, 1.2766146e-12, 8.623173e-06, 2.227816e-09, 2.4487455e-05, 1.8590865e-12, 2.0845388e-11, 0.9932413] 25
1/1 [==============================] - 0s 67ms/step
[3.987743e-09, 0.0003832957, 6.1114406e-07, 1.08785126e-13, 3.805284e-12, 1.2530087e-11, 2.46628e-07, 9.310998e-09, 4.4757815e-09, 2.1373668e-11, 8.2450117e-13, 5.544682e-14, 2.2003077e-14, 0.00028956786, 2.088614e-09, 1.1490844e-11, 1.1135529e-10, 0.0035827742, 2.3943057e-13, 2.3715006e-12, 2.5237983e-05, 6.5422793e-12, 2.8109382e-06, 2.2876332e-12, 2.2471421e-12, 0.99571544] 25
1/1 [==============================] - 0s 77ms/step
[2.301652e-09, 7.659374e-05, 5.8398637e-08, 1.30963355e-14, 3.949252e-14, 1.26542

1/1 [==============================] - 0s 74ms/step
[5.7810624e-08, 0.0004182151, 5.390275e-10, 7.0910996e-14, 1.1405031e-09, 5.0251768e-11, 2.002116e-08, 2.5150283e-07, 6.9357986e-10, 4.8391883e-14, 4.9163958e-11, 2.032763e-13, 8.120431e-16, 1.4069884e-05, 7.4677625e-12, 5.5540697e-16, 5.6997887e-12, 0.00019814687, 2.4393787e-11, 3.6140548e-06, 0.9953726, 6.213464e-10, 8.3368175e-09, 4.7826144e-12, 6.157947e-12, 0.00399289] 20
1/1 [==============================] - 0s 87ms/step
[1.011925e-08, 4.6273053e-05, 1.8199993e-10, 4.1537788e-13, 7.4271866e-10, 3.1249676e-11, 4.685984e-10, 1.13843186e-07, 1.284316e-10, 1.6226944e-14, 1.2930293e-10, 1.4956611e-13, 3.5232174e-16, 6.833579e-08, 9.735242e-13, 1.5711717e-17, 6.9968017e-13, 5.007606e-05, 2.0658035e-12, 5.0204513e-08, 0.9998841, 2.5184502e-11, 4.3296436e-11, 4.381996e-13, 2.7745387e-13, 1.9328627e-05] 20
1/1 [==============================] - 0s 76ms/step
[1.1634629e-08, 9.2813694e-05, 6.8898687e-10, 3.1704895e-12, 5.4650777e-09, 2.21

1/1 [==============================] - 0s 78ms/step
[6.970745e-08, 1.5032742e-15, 5.2580683e-13, 1.5007875e-15, 8.037201e-19, 2.5184969e-25, 3.8892317e-06, 8.266374e-10, 9.623022e-10, 4.447909e-09, 1.266917e-16, 7.676807e-11, 2.2995954e-10, 5.4569865e-19, 1.6362821e-16, 4.5603198e-07, 6.5316696e-11, 1.4765025e-12, 8.073804e-18, 1.7211192e-17, 3.2966762e-18, 2.1877208e-13, 6.2974113e-19, 1.2088217e-17, 0.9999956, 1.6618824e-08] 24


AttributeError: 'NoneType' object has no attribute 'copy'